In [1]:
# Importing Libraries
import pandas as pd

In [2]:
# Adjusting pandas dataframe settings
pd.set_option('display.max_columns', 500)

In [4]:
# Reading Dataframes
books = pd.read_csv("archive/Books.csv", low_memory='False')
ratings = pd.read_csv("archive/Ratings.csv", low_memory='False')
users = pd.read_csv("archive/Users.csv", low_memory='False')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Merging Dataframes
temp_df = books.merge(ratings, how='left', on='ISBN')
final_df = temp_df.merge(users, how='left', on='User-ID')
final_df.shape

(1032345, 12)

['A Time to Kill',
 "Bridget Jones's Diary",
 'Fahrenheit 451',
 'Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Order of the Phoenix (Book 5)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)',
 "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 'House of Sand and Fog',
 "Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)",
 'Midwives: A Novel',
 'Snow Falling on Cedars',
 'The Alienist',
 'The Brethren',
 'The Client',
 'The Divine Secrets of the Ya-Ya Sisterhood: A Novel',
 'The Fellowship of the Ring (The Lord of the Rings, Part 1)',
 "The Hitchhiker's Guide to the Galaxy",
 'The Joy Luck Club',
 'The Pelican Brief',
 'The Testament',
 'When the Wind Blows',
 "White Oleander : A Novel (Oprah's Book Club)"]

In [144]:
# convert dataframe to json
user_df.to_json('user_df.json')

In [8]:
final_df.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0,"stockton, california, usa",18.0


In [9]:
# Creating user book matrix. usning the common books as the rare books are not considered as they would be user specific
def user_book_df(dataframe):
    dataframe.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
    dataframe.dropna(inplace=True)
    dataframe['User-ID'] = dataframe['User-ID'].astype('int')
    final_df = dataframe[dataframe["Book-Rating"] > 0]

    rating_book = pd.DataFrame(final_df['Book-Title'].value_counts())

    rare_books = rating_book[rating_book['Book-Title'] <= 85].index

    # if not in rare books, then title is in common books
    common_books = final_df[~final_df["Book-Title"].isin(rare_books)]
    # pivot table by the user ids for all common books => we get user book matrix with ratings which are given by different users to the books read by our user
    user_book_matrix = common_books.pivot_table(index=['User-ID'], columns=['Book-Title'], values='Book-Rating')
    return user_book_matrix

# store the dataframe, dropping urls,na, and converting user-id to int, and only taking rows with book-rating > 0,
user_book_matrix = user_book_df(final_df)

Now we got a dataframe which has only the common books with user ids as index and the book names as columns and ratings as the values in those columns

In [49]:
user_book_matrix.head(5)

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,A Prayer for Owen Meany,A Time to Kill,A Walk to Remember,ANGELA'S ASHES,American Gods,Angels &amp; Demons,Animal Farm,Balzac and the Little Chinese Seamstress : A Novel,Bel Canto: A Novel,Brave New World,Bridget Jones's Diary,Chocolat,Confessions of a Shopaholic (Summer Display Opportunity),Digital Fortress : A Thriller,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dolores Claiborne,Dreamcatcher,Empire Falls,Ender's Game (Ender Wiggins Saga (Paperback)),Fahrenheit 451,Fast Food Nation: The Dark Side of the All-American Meal,Girl with a Pearl Earring,"Girl, Interrupted",Good in Bed,Hannibal,Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel),Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Book 1),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),High Fidelity,House of Sand and Fog,How to Be Good,Icy Sparks,Interview with the Vampire,Into Thin Air : A Personal Account of the Mt. Everest Disaster,Jurassic Park,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Life of Pi,Little Altars Everywhere: A Novel,Lord of the Flies,Lucky : A Memoir,Me Talk Pretty One Day,Message in a Bottle,Midwives: A Novel,Misery,Mystic River,Neverwhere,Nickel and Dimed: On (Not) Getting By in America,One Door Away from Heaven,Outlander,Red Dragon,SHIPPING NEWS,She's Come Undone (Oprah's Book Club (Paperback)),She's Come Undone (Oprah's Book Club),Silence of the Lambs,Skipping Christmas,Snow Falling on Cedars,Sphere,Stupid White Men ...and Other Sorry Excuses for the State of the Nation!,Summer Sisters,Suzanne's Diary for Nicholas,The Alienist,"The Bad Beginning (A Series of Unfortunate Events, Book 1)",The Beach House,The Bean Trees,The Bonesetter's Daughter,The Brethren,The Bridges of Madison County,The Catcher in the Rye,The Chamber,The Cider House Rules,The Client,The Color Purple,The Da Vinci Code,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Firm,The Five People You Meet in Heaven,The Girls' Guide to Hunting and Fishing,"The Golden Compass (His Dark Materials, Book 1)",The Great Gatsby,The Handmaid's Tale,The Hitchhiker's Guide to the Galaxy,The Hobbit : The Enchanting Prelude to The Lord of the Rings,The Horse Whisperer,The Joy Luck Club,The King of Torts,The Last Time They Met : A Novel,The Lovely Bones: A Novel,The Nanny Diaries: A Novel,The No. 1 Ladies' Detective Agency (Today Show Book Club #8),The Notebook,The Partner,The Pelican Brief,The Pilot's Wife : A Novel,The Poisonwood Bible: A Novel,The Rainmaker,The Reader,The Red Tent (Bestselling Backlist),The Rescue,The Runaway Jury,The Secret Life of Bees,The Street Lawyer,"The Subtle Knife (His Dark Materials, Book 2)",The Summons,The Testament,"The Two Towers (The Lord of the Rings, Part 2)","The Vampire Lestat (Vampire Chronicles, Book II)",Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [133]:
user_book_matrix.shape

(8742, 129)

## TODO: Need to change this part and take input from the frontend instead

we will ask the user to rate a set number of books from our given dataset 
- take the json
- add input to csv file, in the model continue with the user id
- convert json to a 1d dataframe 
- store the 1d dataframe in user df
#### todo: we will need to append the columns with nan as well so that when we append the other books user ahs not watched till now are also visible to the user
 

In [10]:
# Let's choose a user and recommend books to him/her.
# User Based Collaborative Filtering Engine provide us a recommendation system by using user similarities instead of item similarities.
# In that case, a score is calculated for users according to their ratings and correlations. This calculation can be weighted, that means,
# if you are thinking that the ratings are more important than readers correlations, you can give more weight to ratings. In this case, i will make it like this.
user = int('153662')

In [118]:
user_id = users['User-ID'].max() + 1
user_id

278859

In [136]:
cols= user_book_matrix.columns
len(cols)

129

In [131]:
len(user_book_matrix.columns)

129

In [137]:
data = {
        "Classical Mythology": 5,
        "The Mummies of Urumchi": 4        
    }
# convert into a dataframe with cols  as colums and values as the ratings and index as user id
user_ratingsaman = pd.DataFrame(data, index=[user_id], columns=cols)
user_ratingsaman
# append this to our final dataframe
user_book_matrix = user_book_matrix.append(user_ratingsaman)

In [150]:
# show books read by the user which have non nan ratings


Series([], Name: 278859, dtype: float64)

In [142]:
users.shape

(278858, 3)

In [140]:
user_book_matrix.tail(3)

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,A Prayer for Owen Meany,A Time to Kill,A Walk to Remember,ANGELA'S ASHES,American Gods,Angels &amp; Demons,Animal Farm,Balzac and the Little Chinese Seamstress : A Novel,Bel Canto: A Novel,Brave New World,Bridget Jones's Diary,Chocolat,Confessions of a Shopaholic (Summer Display Opportunity),Digital Fortress : A Thriller,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dolores Claiborne,Dreamcatcher,Empire Falls,Ender's Game (Ender Wiggins Saga (Paperback)),Fahrenheit 451,Fast Food Nation: The Dark Side of the All-American Meal,Girl with a Pearl Earring,"Girl, Interrupted",Good in Bed,Hannibal,Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel),Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Book 1),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),High Fidelity,House of Sand and Fog,How to Be Good,Icy Sparks,Interview with the Vampire,Into Thin Air : A Personal Account of the Mt. Everest Disaster,Jurassic Park,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Life of Pi,Little Altars Everywhere: A Novel,Lord of the Flies,Lucky : A Memoir,Me Talk Pretty One Day,Message in a Bottle,Midwives: A Novel,Misery,Mystic River,Neverwhere,Nickel and Dimed: On (Not) Getting By in America,One Door Away from Heaven,Outlander,Red Dragon,SHIPPING NEWS,She's Come Undone (Oprah's Book Club (Paperback)),She's Come Undone (Oprah's Book Club),Silence of the Lambs,Skipping Christmas,Snow Falling on Cedars,Sphere,Stupid White Men ...and Other Sorry Excuses for the State of the Nation!,Summer Sisters,Suzanne's Diary for Nicholas,The Alienist,"The Bad Beginning (A Series of Unfortunate Events, Book 1)",The Beach House,The Bean Trees,The Bonesetter's Daughter,The Brethren,The Bridges of Madison County,The Catcher in the Rye,The Chamber,The Cider House Rules,The Client,The Color Purple,The Da Vinci Code,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Firm,The Five People You Meet in Heaven,The Girls' Guide to Hunting and Fishing,"The Golden Compass (His Dark Materials, Book 1)",The Great Gatsby,The Handmaid's Tale,The Hitchhiker's Guide to the Galaxy,The Hobbit : The Enchanting Prelude to The Lord of the Rings,The Horse Whisperer,The Joy Luck Club,The King of Torts,The Last Time They Met : A Novel,The Lovely Bones: A Novel,The Nanny Diaries: A Novel,The No. 1 Ladies' Detective Agency (Today Show Book Club #8),The Notebook,The Partner,The Pelican Brief,The Pilot's Wife : A Novel,The Poisonwood Bible: A Novel,The Rainmaker,The Reader,The Red Tent (Bestselling Backlist),The Rescue,The Runaway Jury,The Secret Life of Bees,The Street Lawyer,"The Subtle Knife (His Dark Materials, Book 2)",The Summons,The Testament,"The Two Towers (The Lord of the Rings, Part 2)","The Vampire Lestat (Vampire Chronicles, Book II)",Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
278844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [120]:
user.head(2)

,book1,book2
278859,NaN,NaN


In [113]:
user_book_matrix.head(2)

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,A Prayer for Owen Meany,A Time to Kill,A Walk to Remember,ANGELA'S ASHES,American Gods,Angels &amp; Demons,Animal Farm,Balzac and the Little Chinese Seamstress : A Novel,Bel Canto: A Novel,Brave New World,Bridget Jones's Diary,Chocolat,Confessions of a Shopaholic (Summer Display Opportunity),Digital Fortress : A Thriller,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dolores Claiborne,Dreamcatcher,Empire Falls,Ender's Game (Ender Wiggins Saga (Paperback)),Fahrenheit 451,Fast Food Nation: The Dark Side of the All-American Meal,Girl with a Pearl Earring,"Girl, Interrupted",Good in Bed,Hannibal,Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel),Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Book 1),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),High Fidelity,House of Sand and Fog,How to Be Good,Icy Sparks,Interview with the Vampire,Into Thin Air : A Personal Account of the Mt. Everest Disaster,Jurassic Park,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Life of Pi,Little Altars Everywhere: A Novel,Lord of the Flies,Lucky : A Memoir,Me Talk Pretty One Day,Message in a Bottle,Midwives: A Novel,Misery,Mystic River,Neverwhere,Nickel and Dimed: On (Not) Getting By in America,One Door Away from Heaven,Outlander,Red Dragon,SHIPPING NEWS,She's Come Undone (Oprah's Book Club (Paperback)),She's Come Undone (Oprah's Book Club),Silence of the Lambs,Skipping Christmas,Snow Falling on Cedars,Sphere,Stupid White Men ...and Other Sorry Excuses for the State of the Nation!,Summer Sisters,Suzanne's Diary for Nicholas,The Alienist,"The Bad Beginning (A Series of Unfortunate Events, Book 1)",The Beach House,The Bean Trees,The Bonesetter's Daughter,The Brethren,The Bridges of Madison County,The Catcher in the Rye,The Chamber,The Cider House Rules,The Client,The Color Purple,The Da Vinci Code,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Firm,The Five People You Meet in Heaven,The Girls' Guide to Hunting and Fishing,"The Golden Compass (His Dark Materials, Book 1)",The Great Gatsby,The Handmaid's Tale,The Hitchhiker's Guide to the Galaxy,The Hobbit : The Enchanting Prelude to The Lord of the Rings,The Horse Whisperer,The Joy Luck Club,The King of Torts,The Last Time They Met : A Novel,The Lovely Bones: A Novel,The Nanny Diaries: A Novel,The No. 1 Ladies' Detective Agency (Today Show Book Club #8),The Notebook,The Partner,The Pelican Brief,The Pilot's Wife : A Novel,The Poisonwood Bible: A Novel,The Rainmaker,The Reader,The Red Tent (Bestselling Backlist),The Rescue,The Runaway Jury,The Secret Life of Bees,The Street Lawyer,"The Subtle Knife (His Dark Materials, Book 2)",The Summons,The Testament,"The Two Towers (The Lord of the Rings, Part 2)","The Vampire Lestat (Vampire Chronicles, Book II)",Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [51]:
# Choosing OUR user from user_book_matrix
user_df = user_book_matrix[user_book_matrix.index == user]
user_df

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,A Prayer for Owen Meany,A Time to Kill,A Walk to Remember,ANGELA'S ASHES,American Gods,Angels &amp; Demons,Animal Farm,Balzac and the Little Chinese Seamstress : A Novel,Bel Canto: A Novel,Brave New World,Bridget Jones's Diary,Chocolat,Confessions of a Shopaholic (Summer Display Opportunity),Digital Fortress : A Thriller,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dolores Claiborne,Dreamcatcher,Empire Falls,Ender's Game (Ender Wiggins Saga (Paperback)),Fahrenheit 451,Fast Food Nation: The Dark Side of the All-American Meal,Girl with a Pearl Earring,"Girl, Interrupted",Good in Bed,Hannibal,Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel),Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Book 1),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),High Fidelity,House of Sand and Fog,How to Be Good,Icy Sparks,Interview with the Vampire,Into Thin Air : A Personal Account of the Mt. Everest Disaster,Jurassic Park,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Life of Pi,Little Altars Everywhere: A Novel,Lord of the Flies,Lucky : A Memoir,Me Talk Pretty One Day,Message in a Bottle,Midwives: A Novel,Misery,Mystic River,Neverwhere,Nickel and Dimed: On (Not) Getting By in America,One Door Away from Heaven,Outlander,Red Dragon,SHIPPING NEWS,She's Come Undone (Oprah's Book Club (Paperback)),She's Come Undone (Oprah's Book Club),Silence of the Lambs,Skipping Christmas,Snow Falling on Cedars,Sphere,Stupid White Men ...and Other Sorry Excuses for the State of the Nation!,Summer Sisters,Suzanne's Diary for Nicholas,The Alienist,"The Bad Beginning (A Series of Unfortunate Events, Book 1)",The Beach House,The Bean Trees,The Bonesetter's Daughter,The Brethren,The Bridges of Madison County,The Catcher in the Rye,The Chamber,The Cider House Rules,The Client,The Color Purple,The Da Vinci Code,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Firm,The Five People You Meet in Heaven,The Girls' Guide to Hunting and Fishing,"The Golden Compass (His Dark Materials, Book 1)",The Great Gatsby,The Handmaid's Tale,The Hitchhiker's Guide to the Galaxy,The Hobbit : The Enchanting Prelude to The Lord of the Rings,The Horse Whisperer,The Joy Luck Club,The King of Torts,The Last Time They Met : A Novel,The Lovely Bones: A Novel,The Nanny Diaries: A Novel,The No. 1 Ladies' Detective Agency (Today Show Book Club #8),The Notebook,The Partner,The Pelican Brief,The Pilot's Wife : A Novel,The Poisonwood Bible: A Novel,The Rainmaker,The Reader,The Red Tent (Bestselling Backlist),The Rescue,The Runaway Jury,The Secret Life of Bees,The Street Lawyer,"The Subtle Knife (His Dark Materials, Book 2)",The Summons,The Testament,"The Two Towers (The Lord of the Rings, Part 2)","The Vampire Lestat (Vampire Chronicles, Book II)",Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
153662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,10.0,10.0,NaN,10.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,10.0,N

In [92]:
# Taking read books by user.
user_read_books = user_df.columns[user_df.notna().any()].tolist()
print(user_read_books, '\n',len(user_read_books))

['A Time to Kill', "Bridget Jones's Diary", 'Fahrenheit 451', 'Harry Potter and the Chamber of Secrets (Book 2)', 'Harry Potter and the Goblet of Fire (Book 4)', 'Harry Potter and the Order of the Phoenix (Book 5)', 'Harry Potter and the Prisoner of Azkaban (Book 3)', "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))", 'House of Sand and Fog', "Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)", 'Midwives: A Novel', 'Snow Falling on Cedars', 'The Alienist', 'The Brethren', 'The Client', 'The Divine Secrets of the Ya-Ya Sisterhood: A Novel', 'The Fellowship of the Ring (The Lord of the Rings, Part 1)', "The Hitchhiker's Guide to the Galaxy", 'The Joy Luck Club', 'The Pelican Brief', 'The Testament', 'When the Wind Blows', "White Oleander : A Novel (Oprah's Book Club)"] 
 23


In [13]:
# How many books are read by user?
len(user_read_books) # 23

23

In [52]:
# Choosing users book with all users.
read_books = user_book_matrix[user_read_books]
read_books.head(5)
# read books is a dataframe storing the names of all the books that are read by our usernand their ratings respectively

Book-Title,A Time to Kill,Bridget Jones's Diary,Fahrenheit 451,Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),House of Sand and Fog,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Midwives: A Novel,Snow Falling on Cedars,The Alienist,The Brethren,The Client,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Hitchhiker's Guide to the Galaxy,The Joy Luck Club,The Pelican Brief,The Testament,When the Wind Blows,White Oleander : A Novel (Oprah's Book Club)
User-ID,,,,,,,,,,,,,,,,,,,,,,,
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# reset the index of the dataframe and store the total ratings given by each user in the read books
user_book_count_df = read_books.T.notnull().sum().reset_index()
# here we 
user_book_count_df.columns = ['User-ID','Count']
user_book_count_df['Count'].describe(percentiles=(0.1,0.25,0.4,0.6,0.75,0.9))

count    8742.000000
mean        0.392473
std         0.884887
min         0.000000
10%         0.000000
25%         0.000000
40%         0.000000
50%         0.000000
60%         0.000000
75%         1.000000
90%         1.000000
max        23.000000
Name: Count, dtype: float64

In [58]:
print(user_book_count_df.head(10))
print(user_book_count_df.shape)
# 4 would mean that this user has read 4 common books as our user

   User-ID  Count
0       51      0
1      114      0
2      165      0
3      183      1
4      242      1
5      254      4
6      388      0
7      424      0
8      433      1
9      486      0
(8742, 2)


In [36]:
user_book_count_df[user_book_count_df['Count'] > 5].shape

(32, 2)

In [39]:
# Choosing same readers with user having read >5 common books,storing their uids
same_readers_with_user = user_book_count_df[user_book_count_df['Count'] > 5]['User-ID']
same_readers_with_user.head(2)

205     6563
514    16795
Name: User-ID, dtype: int64

In [65]:
# user_book_count_df[user_book_count_df['Count'] > 5][:2]
same_readers_with_user.values

array([  6563,  16795,  21576,  22625,  23872,  30711,  36003,  55492,
        60244,  66942,  85993,  95359, 101851, 102647, 102702, 105517,
       114368, 125519, 136491, 137589, 153662, 159506, 165308, 179734,
       185176, 208815, 220688, 229741, 238889, 240567, 242083, 248477])

In [97]:
# value counts
# weird?
read_books[read_books.index.isin(same_readers_with_user)].head(2)

Book-Title,A Time to Kill,Bridget Jones's Diary,Fahrenheit 451,Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),House of Sand and Fog,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Midwives: A Novel,Snow Falling on Cedars,The Alienist,The Brethren,The Client,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Hitchhiker's Guide to the Galaxy,The Joy Luck Club,The Pelican Brief,The Testament,When the Wind Blows,White Oleander : A Novel (Oprah's Book Club)
User-ID,,,,,,,,,,,,,,,,,,,,,,,
6563,10.0,NaN,NaN,NaN,9.0,10.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
16795,NaN,7.0,8.0,NaN,NaN,NaN,NaN,NaN,9.0,9.0,7.0,NaN,NaN,NaN,5.0,NaN,8.0,NaN,NaN,6.0,NaN,8.0,NaN


In [98]:
user_df[user_read_books]

Book-Title,A Time to Kill,Bridget Jones's Diary,Fahrenheit 451,Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),House of Sand and Fog,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Midwives: A Novel,Snow Falling on Cedars,The Alienist,The Brethren,The Client,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Hitchhiker's Guide to the Galaxy,The Joy Luck Club,The Pelican Brief,The Testament,When the Wind Blows,White Oleander : A Novel (Oprah's Book Club)
User-ID,,,,,,,,,,,,,,,,,,,,,,,
153662,10.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,5.0,9.0,9.0,10.0,10.0,5.0,10.0,10.0,10.0,10.0,9.0,9.0,9.0,10.0,9.0


In [96]:
print(rec_df.shape, '\n', read_books[read_books.index.isin(same_readers_with_user)].shape)

(33, 23) 
 (32, 23)


In [152]:
# convert dataframe rec_df.tail(1) to json
rec_df.tail(1).to_json('rec_df.json')


In [40]:
# adding our user to the datafram with other user who read same book
rec_df = pd.concat([read_books[read_books.index.isin(same_readers_with_user)],user_df[user_read_books]])
rec_df.head(2)

Book-Title,A Time to Kill,Bridget Jones's Diary,Fahrenheit 451,Harry Potter and the Chamber of Secrets (Book 2),Harry Potter and the Goblet of Fire (Book 4),Harry Potter and the Order of the Phoenix (Book 5),Harry Potter and the Prisoner of Azkaban (Book 3),Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),House of Sand and Fog,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),Midwives: A Novel,Snow Falling on Cedars,The Alienist,The Brethren,The Client,The Divine Secrets of the Ya-Ya Sisterhood: A Novel,"The Fellowship of the Ring (The Lord of the Rings, Part 1)",The Hitchhiker's Guide to the Galaxy,The Joy Luck Club,The Pelican Brief,The Testament,When the Wind Blows,White Oleander : A Novel (Oprah's Book Club)
User-ID,,,,,,,,,,,,,,,,,,,,,,,
6563,10.0,NaN,NaN,NaN,9.0,10.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
16795,NaN,7.0,8.0,NaN,NaN,NaN,NaN,NaN,9.0,9.0,7.0,NaN,NaN,NaN,5.0,NaN,8.0,NaN,NaN,6.0,NaN,8.0,NaN


In [103]:
rec_df.T.corr().unstack().sort_values(ascending=False)

User-ID  User-ID
55492    66942      1.0
66942    55492      1.0
153662   102702     1.0
         102702     1.0
102702   153662     1.0
                   ... 
153662   6563       NaN
         55492      NaN
         136491     NaN
         185176     NaN
         208815     NaN
Length: 1089, dtype: float64

In [26]:
# We should take correlations between users, that's why users that in the rec_df have to change from index to columns.
# we take the ids of the user having the highest correlation and drop the duplicates
correlation_df = rec_df.T.corr().unstack().sort_values(ascending=False).drop_duplicates()
correlation_df = pd.DataFrame(correlation_df,columns=['correlation'])
correlation_df.index.names = ['User-ID-1','User-ID-2']
correlation_df = correlation_df.reset_index()
correlation_df.head(10)


,User-ID-1,User-ID-2,correlation
0,55492,66942,1.000000
1,153662,102702,1.000000
2,238889,220688,1.000000
3,105517,105517,1.000000
4,137589,85993,1.000000
5,137589,185176,1.000000
6,6563,125519,0.981981
7,153662,22625,0.979796
8,21576,159506,0.970725
9,242083,248477,0.958706


In [27]:
# We can choose users with more than 0.70 correlation with selected user.
similar_users = correlation_df[(correlation_df['correlation'] >= 0.70) & (correlation_df['User-ID-1'] == user)][['User-ID-2','correlation']]
similar_users = similar_users.reset_index(drop=True)
similar_users.rename(columns={'User-ID-2':'User-ID',
                              'correlation':'Correlations'}, inplace=True)
similar_users

,User-ID,Correlations
0,102702,1.000000
1,22625,0.979796
2,238889,0.846332


In [106]:
# You can reach the correlations between our user which has User-ID (153662) and other readers.
#       User-ID    Correlation
# 0     102702     1.000000
# 1      22625     0.979796
# 2     238889     0.846332

# In this case, our user has 153662 User-ID and you can reach similar readers with our user by using correlation each others.
# However this may not be good recommendation engine for the system since we did not calculate ratings. What i am trying to say,
# for example, a reader can be good correlate with our user, but it maybe voted low rating points. Therefore, we should take into account also ratings.

# Hence, we can recommend good rating books to readers.

# IN SHORT: we found the our correlated readers, BUT since we are recommending we dont want to recommend low rated books => we only recommend good books

# So, we should calculate Weighted Score.

similar_users_df = similar_users.merge(ratings, how='inner',on='User-ID')
similar_users_df = similar_users_df.merge(books,how='inner', on='ISBN')
similar_users_df = similar_users_df[similar_users_df['User-ID'] != user]

# Calculation Weighted Score (a weighting score I made just to make fair observation here, the correlations are given lessr value that the book ratings) book ratings can at max be 5
similar_users_df['Weighted Score'] = (0.4 * similar_users_df['Correlations'] + 0.6 * (similar_users_df['Book-Rating'] / 10)) / 2

In [29]:
similar_users_df.sort_values(by='Weighted Score',ascending=False)

similar_users_df.groupby('User-ID').agg({'Weighted Score':'mean'})
#         Weighted Score
# User-ID
# 22625          0.313127
# 102702         0.344213
# 238889         0.411321

# We can say that the 238889 User-ID has most recommended reader with our user on average.

,Weighted Score
User-ID,
22625,0.313127
102702,0.344213
238889,0.411321


In [109]:
# Finally, we can calculate the weighted score on books. Additionally, a threshold has to be determined for the score, in this case 8.5 => 0.4,
# so that the recommender system do not recommend to the users all of the books which have scores.
recommendation_df = similar_users_df.groupby('Book-Title').agg({'Weighted Score':'mean'}).reset_index()
recommendation_df_by_score = recommendation_df[recommendation_df["Weighted Score"] > 0.40].sort_values(by='Weighted Score',ascending=False)
recommendation_df_by_score = recommendation_df_by_score.merge(books, how='inner',on='Book-Title')
recommendation_df_by_score.tail(5)

,Book-Title,Weighted Score,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
585,Saints,0.409266,0812581407,Orson Scott Card,1988,Tor Books,http://images.amazon.com/images/P/0812581407.0...,http://images.amazon.com/images/P/0812581407.0...,http://images.amazon.com/images/P/0812581407.0...
586,Saints,0.409266,0312876068,Orson Scott Card,2001,Forge,http://images.amazon.com/images/P/0312876068.0...,http://images.amazon.com/images/P/0312876068.0...,http://images.amazon.com/images/P/0312876068.0...
587,Kindred (Black Women Writers Series),0.405959,0807083054,Octavia Butler,1988,Beacon Press,http://images.amazon.com/images/P/0807083054.0...,http://images.amazon.com/images/P/0807083054.0...,http://images.amazon.com/images/P/0807083054.0...
588,A Monk Swimming : A Memoir,0.405959,0786884142,Malachy McCourt,1999,Hyperion,http://images.amazon.com/images/P/0786884142.0...,http://images.amazon.com/images/P/0786884142.0...,http://images.amazon.com/images/P/0786884142.0...
589,CHOCOLATE FOR A WOMAN'S SPIRIT : 77 STORIES OF...,0.405959,068484897X,Kay Allenbaugh,1999,Fireside,http://images.amazon.com/images/P/068484897X.0...,http://images.amazon.com/images/P/068484897X.0...,http://images.amazon.com/images/P/068484897X.0...


In [154]:
# make a dictionary with book titles as keys and the Image-URL-M from recommendation_df_by_score
rdict ={}
for index, row in recommendation_df_by_score.iterrows():
    rdict[row['Book-Title']] = row['Image-URL-M']
    




In [157]:
rdict

{"Frisco's Kid": 'http://images.amazon.com/images/P/1551667592.01.MZZZZZZZ.jpg',
 'The Dragon Queen (Tales of Guinevere, Book 1)': 'http://images.amazon.com/images/P/0345443993.01.MZZZZZZZ.jpg',
 'Hot Chocolate': 'http://images.amazon.com/images/P/0515124524.01.MZZZZZZZ.jpg',
 'The Defiant Hero': 'http://images.amazon.com/images/P/0804119538.01.MZZZZZZZ.jpg',
 'Hot on Her Tail (Harlequin Blaze, No 43)': 'http://images.amazon.com/images/P/0373790473.01.MZZZZZZZ.jpg',
 'Hunting Midnight (Berkley Sensation)': 'http://images.amazon.com/images/P/0425193039.01.MZZZZZZZ.jpg',
 'I Brake for Bad Boys': 'http://images.amazon.com/images/P/0758204175.01.MZZZZZZZ.jpg',
 'I Love Bad Boys': 'http://images.amazon.com/images/P/0758201346.01.MZZZZZZZ.jpg',
 'I Love Rogues': 'http://images.amazon.com/images/P/0758204191.01.MZZZZZZZ.jpg',
 'The Claiming of Sleeping Beauty (Sleeping Beauty)': 'http://images.amazon.com/images/P/0452281423.01.MZZZZZZZ.jpg',
 'Impetuous': 'http://images.amazon.com/images/P/03

In [31]:
# Lastly, we can determine a list to will be recommended.
recommended_list = recommendation_df_by_score.sort_values(by='Weighted Score',ascending=False).drop_duplicates(subset='Book-Title')['Book-Title'].head(10).to_list()
recommended_list

["Frisco's Kid",
 'Blue Heaven, Black Night (Zebra Regency Romance)',
 'Caught In The Act',
 'The Wilde Side',
 'Unexpected',
 'Wanton (Blaze) (Harlequin Temptation, No. 752)',
 'American Heroes: Riley',
 'Dark Gold',
 'Conquer the Night',
 'Dance With The Devil: A Dark-Hunter Novel']